## Data Preparation

In [1]:
from typing import Optional
from dataclasses import dataclass
import pickle
from datetime import datetime as dt
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import os

### Necesarry Helper Functions

In [37]:
@dataclass
class Form:
    # Same for both sources
    filing_date: str
    trade_date: str
    ticker: str
    price: float
    qty_bought: float
    qty_owned: float

    #Source: openinsider.com
    insider_name_link: Optional[str] = None 
    company_link: Optional[str] = None
    buyer_title: Optional[str] = None
    
    #Source: SEC Bulk Data
    accession_number: Optional[str] = None
    insider_name: Optional[str] = None 
    company_name: Optional[str] = None
    insider_relationship: Optional[str] = None 
    insider_title: Optional[str] = None 
    fd_index: Optional[int] = None

    def __eq__(self, other) -> bool:
        return self.filing_date == other.filing_date and self.trade_date == other.trade_date and self.ticker == other.ticker and self.price == other.price and self.qty_bought == other.qty_bought and self.qty_owned == self.qty_owned
    
    def __ne__(self, other) -> bool:
        return not self.__eq__(other)
    
    def delta_own(self):
        if self.qty_owned is None or self.qty_bought is None: raise ValueError("Class values were None")

        if self.qty_owned - self.qty_bought == 0:
            return 100
        else:
            return float(self.qty_bought / (self.qty_owned - self.qty_bought)) * 100

    def days_ago(self) -> int:
        if self.filing_date is None or self.trade_date is None:
            raise ValueError("Class values were None")
        fd = dt.strptime(self.filing_date, "%Y-%m-%d")
        td = dt.strptime(self.trade_date,  "%Y-%m-%d")
        return (fd - td).days
  
def save_data(data, fname):
    """
    base path is Assets folder
    """
    with open(f"src/OpenInsider/Assets/{fname}.pkl", "wb") as file:
        pickle.dump(data, file)

def load_data(fname) -> list:
    with open(f"Assets/{fname}.pkl", "rb") as file:
        data = pickle.load(file)
        return data

def tickers_from_data(forms: list[Form]):
    tickers = {}
    for form in forms:
        tickers[form.ticker] = 1

    return [key for key in tickers.keys()]

@dataclass
class Candle:
    date: str
    o: float
    c: float
    h: float
    l: float
    v: int

@dataclass
class Trade:
    form: Form
    candles: list[Candle]
    onem_profit: Optional[float] = None
    twom_profit: Optional[float] = None
    threem_profit: Optional[float] = None

    def catalyst_size(self, shift):
        """
        NOTE: candles[0] needs to be at the filing date, and candles[-1] should be the final day\n
        @param n_days is how many days after the filing date the calculations should consider\n
        @param shift is how many days each peak should be shifted to the right by
        """
        avg_peak_val: float = 0.0

        closes = [c.c for c in self.candles]
        _, peaks = get_peaks(closes, shift)
        
        if len(peaks) != 0:
            for peak in peaks:
                avg_peak_val += self.candles[peak].c
                # print(f"{self.candles[peak].c}, {peak}")
            avg_peak_val /= len(peaks)
            # print(f"AVG PEAK: {avg_peak_val}")
            # print(f"First Close: {self.candles[0].c}")
            return pct_change(self.candles[0].c, avg_peak_val)
        
        # if there are no peaks (sad face), return absolute change
        return pct_change(closes[0], closes[-1])
    
    def returns(self, days: int, candles: list[Candle] = []) -> float:
        """
        Percent return over (@param days) number of days from close to close\n
        Negative days shows past returns up to the filing date\n
        Used to check the passivity of a certain transaction\n
        """
        if days > 0:
            if len(self.candles) > days:
                return pct_change(self.candles[0].c, self.candles[days - 1].c)
            
        data = []
        if len(candles) == 0:
            data = get_ticker_data(self.form.ticker)
            assert data is not None
        else: 
            data = candles

        fdi = self.form.fd_index
        assert fdi is not None
    
        if days < 0:
            data = trim_ticker_data(data, fdi, (-days, 1))
        elif days > 0:
            data = trim_ticker_data(data, fdi, (0, days))
        else: return 0

        return pct_change(data[0].c, data[-1].c)
    
def moving_avg(data: list, length: int):
    new_data = []
    window = []
    for d in data:
        if len(window) <= length:
            window.append(d)
        else:
            window = window[1:]
            window.append(d)

        new_data.append(sum(window)/len(window))
    return new_data

def pct_change(from_, to_):
    return ((to_ - from_) / from_) * 100

def get_peaks(price_data: list, shift: int):
    """
    @param price_data: list of prices (typically the closing prices)\n
    @param shift: move each peak to the right n-days because seldom will you exit at the true peak of a stock\n
    """
    ma = moving_avg(price_data, 7)

    dist = 7
    height_multiplier = 1.05
    peaks, _ = find_peaks(ma, distance=dist, height=ma[0] * height_multiplier, width=5) # one week between peaks and peak must be a 5% increase from day 0


    while len(peaks) == 0 and dist >= 1:
        peaks, _ = find_peaks(ma, distance=dist, width=5) # one week between peaks and peak must be a 5% increase from day 0
        dist -= 1

    if pct_change(ma[0], ma[-1]) > 5:
        peaks = list(peaks)

        s = 0.0
        for p in peaks:
            s += ma[p]

        avg_peak_height = s/len(peaks)
        if ma[-2] > avg_peak_height:
            peaks.append(len(ma) - 2) # add second index from the end

    if len(peaks) == 0: return [], []

    for i in range(len(peaks)):
        if i < len(peaks) - 1:
            peaks[i] += shift 

    return ma, peaks

def show_trade(trade: Trade, show_peaks: bool):
    prices = pd.DataFrame({
        "high"  : [candle.h  for candle in trade.candles],
        "low"   : [candle.l  for candle in trade.candles],
        "open"  : [candle.o  for candle in trade.candles],
        "close" : [candle.c  for candle in trade.candles]
    })
    green  = prices[prices.close >= prices.open] # green candles
    red    = prices[prices.close < prices.open] # red candles
    w1, w2 = 0.4, 0.02 # width of thick part and width of extrema

    _, ax = plt.subplots()
    # graph green candles (x, height, width, bottom, color)
    ax.bar(green.index, green.close - green.open, w1, green.open, color='green') # thick middle part
    ax.bar(green.index, green.high  - green.close, w2, green.close, color='black') # high price
    ax.bar(green.index, green.low  - green.open, w2, green.open, color='black') # low price
    
    ax.bar(red.index, red.close - red.open, w1, red.open, color='red') # thick middle part
    ax.bar(red.index, red.high  - red.open, w2, red.open, color='black') # high price
    ax.bar(red.index, red.low   - red.close, w2, red.close, color='black') # low price

    if show_peaks:
        ma, peaks = get_peaks(list(prices.close.array), 3)
        ax.plot(ma)
        
        for peak in peaks:
            ax.plot(peak, list(prices.close.array)[peak], "bo")


    title = f"{trade.form.ticker} @ {trade.candles[0].date} (FD: {trade.form.filing_date}, TD: {trade.form.trade_date})"

    ax.set_title(title)
    plt.show()

def get_ticker_data(ticker: str):
    """
    open and return the data associated with a certain ticker\n
    returns None is no data are found
    """
    contents = os.listdir(f"Assets/Historical_Stock_Data")
    if f"{ticker}.pkl" not in contents: return None
    else: return load_data(f"Historical_Stock_Data/{ticker}")

def trim_ticker_data(data: list[Candle], fd_index: int, window: tuple[int, int]):
    """
    @param window is a tuple of how many candles should be included before and after the filing date\n
    \t --> i.e. (10, 100) means to show from 10 days before to 100 days after
    """
    start = fd_index - window[0] if fd_index - window[0] >= 0 else 0
    end = fd_index + window[1] if fd_index + window[1] < len(data) else len(data) - 1
    data = data[start : end]
    return data

def trade_from_histform(form: Form, n_days):
    """
    returns trade object with candles from fd up to (fd + n_days)
    """
    data = get_ticker_data(form.ticker)
    if data is None: return

    i = form.fd_index
    if i is None: 
        print("❌FDI NOT FOUND")
        return

    data = data[i:i + n_days]

    trade = Trade(
        form,
        data
    )

    return trade

def show_hist_trade(form: Form, n_days: int, show_peaks: bool):
    if form.ticker is None: return
    if form.filing_date is None: return

    data = get_ticker_data(form.ticker)
    if data is None: return

    i = form.fd_index
    if i is None: 
        print("❌FDI NOT FOUND")
        return

    data = data[i : i + n_days]

    trade = Trade(
        form,
        data
    )

    show_trade(trade, show_peaks)

    return trade

### Load in Data

In [36]:
forms: list[Form] = load_data("HistForms")

In [60]:
l = len(forms)
trades: list[Trade] = []
for i, form in enumerate(forms):
    print(f"{i/(l - 1)*100:.2f}%", end='\r')
    t = trade_from_histform(form, 100)
    if t is not None: # * this line ensures each trade has data!
        trades.append(t)

trades = list(filter(lambda x: len(x.candles) > 0 and x.candles is not None, trades)) # filter such that...

### Ensure Successful Loading

In [4]:
trade = show_hist_trade(trades[10].form, 100, False) # set result to a variable so it doesn't print it out

NameError: name 'trades' is not defined

In [ ]:
trades[10].returns(30)

In [ ]:
trades[10].returns(60)

In [ ]:
trades[10].returns(90)

### Track the Insider and Company in Question

In [63]:
def get_insider_and_company_dict(trades: list[Trade]) -> tuple[dict[str, list[Trade]], dict[str, list[Trade]]]:
    # get dictionary of "insider name" : [forms...]
    insider_dict: dict[str, list[Trade]] = {}
    # get dictionary of "ticker" : [forms...]
    company_dict: dict[str, list[Trade]] = {}

    for trade in trades:
        iname = trade.form.insider_name
        if iname is None: continue
        if iname not in insider_dict:
            insider_dict[iname] = [trade]
        else:
            insider_dict[iname].append(trade)

        cname = trade.form.ticker
        if cname is None: continue
        if cname not in company_dict:
            company_dict[cname] = [trade]
        else:
            company_dict[cname].append(trade)
        
    return insider_dict, company_dict

In [64]:
def get_cluster_score(curr: Trade, company_trades: list[Trade]):
    cd = dt.strptime(curr.form.trade_date, "%Y-%m-%d")

    cluster_score = 0
    for t in company_trades:
        td = dt.strptime(t.form.trade_date, "%Y-%m-%d")
        if (cd - td).days < 30:
            cluster_score += 1
    
    return cluster_score

### Map Insider Job Title to Numbers

- Other = 1
- 10% owners = 2
- Director = 3
- Officer = 4
- Vice President (EVP or SVP) = 5
- President = 6

In [65]:
def map_insider_title(f: Form) -> int:
    r = f.insider_relationship
    assert r is not None
    r = r.lower()

    if "other" in r:
        return 1

    if "tenpercentowner" in r:
        return 2
    
    if "director" in r:
        return 3
    
    if "officer" in r:
        t = f.insider_title
        assert t is not None
        t = t.lower()

        if "vice president" in t or "svp" in t or "evp" in t:
            # must be before the check for president
            return 5
        
        if "president" in t:
            return 6
        
        return 4

    #* tested on all forms and never reached
    return 0

### Determine Output Profit Ranges

In [14]:
import numpy as np

data = []

l = len(trades)
for i, t in enumerate(trades):
    data.append(
        sum([t.returns(30), t.returns(60), t.returns(90)])/3
    )

median = np.median(data)
q3 = np.percentile(data, 75)
q1 = np.percentile(data, 25)

iqr = q3 - q1
upper_outlier = q3+1.5*iqr
lower_outlier = q1-1.5*iqr
upper_whisker = max(list(filter(lambda x: x <= upper_outlier, data)))
lower_whisker = min(list(filter(lambda x: x >= lower_outlier, data)))

##### Output Profit Ranges (%)

- Outlier Profit = >50
- Big Profit = (14, 50]
- Moderate Profit = (1-14]
- Stagnation = [-1, 1]
- Moderate Loss = (-1, -10]
- Big Loss = (-10, -46]
- Outlier Loss = <-46

## Neural Network Development

In [66]:
import tensorflow as tf
keras = tf.keras

In [ ]:
i_dict, c_dict = get_insider_and_company_dict(trades)

data = []
labels = []
l = len(trades)
for i, trade in enumerate(trades):
    print(f"{i/(l - 1)*100: .2f}%", end="\r")
    try:
        name = trade.form.insider_name
        if name is None: continue

        profit = trade.returns(90)

        i_profits = [(trade.form.qty_bought * trade.form.price) * profit/100 for trade in i_dict[name]]
        wins = list(filter(lambda x: x > 0, i_profits))
        losses = list(filter(lambda x: x < 0, i_profits))
        
        row = [
            trade.form.qty_bought, trade.form.price, trade.form.delta_own(), map_insider_title(trade.form),
            trade.returns(-30), get_cluster_score(trade, c_dict[trade.form.ticker]), trade.form.days_ago(),    
        ]
        if len(i_profits) > 0:
            row.append(sum(i_profits)/len(i_profits))
        else:
            row.append(0)

        # if len(wins) > 0:
        #     row.append(sum(wins)/len(wins))
        #     row.append(len(wins))
        # else:
        #     row.append(0)
        #     row.append(0)

        # if len(losses) > 0:
        #     row.append(sum(losses)/len(losses))
        #     row.append(len(losses))
        # else:
        #     row.append(0)
        #     row.append(0)

        data.append(row)

        zeros = np.zeros(7)
        if profit > 50:
            zeros[0] = 1
        elif profit > 14 and profit <= 50:
            zeros[1] = 1
        elif profit > 1 and profit <= 14:
            zeros[2] = 1
        elif profit >= -1 and profit <= 1:
            zeros[3] = 1
        elif profit < -1 and profit >= -10:
            zeros[4] = 1
        elif profit < -10 and profit >= -46:
            zeros[5] = 1
        elif profit < -46:
            zeros[6] =1

        labels.append(zeros)
    except Exception as err:
        print(f"{type(err)} @ {i}")

#### Data Structure
- Number of shares bought
- Price per share
- Percent change in ownership
- Insider position
- Previous 30 day returns
- Cluster score
- Days between filing and trading
- Average profit of insider's previous transactions

In [68]:
data[4]

[38124.54,
 19.06,
 -105.5364026780687,
 3,
 131.62790697674419,
 22,
 2,
 -14259.99319051245]

In [72]:
import math
train_data, train_labels = np.array(data[:math.floor(len(data)*5/6)]), np.array(labels[:math.floor(len(data)*5/6)])
test_data, test_labels = np.array(data[math.floor(len(data)*5/6):]), np.array(labels[math.floor(len(data)*5/6):])

In [73]:
train_data.shape

(31369, 8)

In [74]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # hide warnings

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(train_data.shape[1],)),
    tf.keras.layers.Dense(100, activation='tanh'),
    tf.keras.layers.Dense(50, activation='tanh'),
    tf.keras.layers.Dense(7)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredLogarithmicError(),
    metrics=['accuracy']
)

In [75]:
history = model.fit(train_data, train_labels, epochs=1, batch_size=1)

31369/31369 [==============================] - 50s 2ms/step - loss: 0.0521 - accuracy: 0.4697


In [76]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(test_acc)

197/197 [==============================] - 1s 2ms/step - loss: 0.0483 - accuracy: 0.4901
0.4901179373264313


In [ ]:
model.save("Models/First")

In [82]:
model = keras.models.load_model("Models/First")
if model is None: raise ValueError()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               900       
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 7)                 357       
                                                                 
Total params: 6,307
Trainable params: 6,307
Non-trainable params: 0
_________________________________________________________________
